### Introduction

The project aims to build a model to predict drug efficacy of molecules.

A pre-processed HIV dataset with 3 classess (CA - Confirmed active, CM - Confirmed moderately active, CI - Confirmed inactive and benign) is available [here](http://moleculenet.ai/datasets-1). The raw data is available [here](https://wiki.nci.nih.gov/display/NCIDTPdata/AIDS+Antiviral+Screen+Data)
We perform stratified random splitting on the dataset: 80% of the images are in train set and 20% of the images are in test set.

#### Import required library

In [9]:
import os
import sys
sys.path.insert(0, os.getcwd())
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import config_context
from config import Config
#from utils.data import read_data, get_prediction_score
from models import feature_extraction
import pickle
import warnings
import argparse


ModuleNotFoundError: No module named 'models'

In [10]:
print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

NameError: name 'tf' is not defined

In [11]:
import torch
#import torchvision
torch.zeros(1).cuda()

ModuleNotFoundError: No module named 'torch'

In [4]:
x = torch.rand(5,3)
print(x)

tensor([[0.8549, 0.0748, 0.7328],
        [0.8880, 0.7631, 0.3456],
        [0.3080, 0.2034, 0.3715],
        [0.0315, 0.7793, 0.1140],
        [0.5867, 0.3209, 0.7414]])


In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available)

ModuleNotFoundError: No module named 'torch'

In [1]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [2]:
import tensorflow